## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [10]:
import mlflow
import os
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from mlflow.tracking import MlflowClient


In [3]:
os.environ["AWS_PROFILE"] = "ArunG" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_SERVER_HOST)
mlflow.set_experiment("green-taxi-duration")

#TRACKING_SERVER_HOST = "ec2-13-60-29-159.eu-north-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
#mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

2024/06/22 20:17:01 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-gansi/1', creation_time=1719067621678, experiment_id='1', last_update_time=1719067621678, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'", TRACKING_SERVER_HOST)

tracking URI: 'http://127.0.0.1:5000' http://127.0.0.1:5000


In [37]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [38]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [40]:
with mlflow.start_run():
   params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
   mlflow.log_params(params)

   dv = DictVectorizer()
   model = RandomForestRegressor(**params, n_jobs=-1)

   X_train = dv.fit_transform(dict_train)
   
   model.fit(X_train,y_train)
    
   X_val = dv.transform(dict_val)
   y_pred = model.predict(X_val)

   rmse = mean_squared_error(y_pred, y_val, squared=False)
   print(params, rmse)
   mlflow.log_metric('rmse', rmse)

   mlflow.sklearn.log_model(model, artifact_path="model")
    
   with open('dict_vectorizer.bin','wb') as f_out:
      pickle.dump(dv,f_out)
    
   mlflow.log_artifact('dict_vectorizer.bin')

  (0, 9531)	1.0
  (0, 13220)	1.01
  (1, 2841)	1.0
  (1, 13220)	2.53
  (2, 9317)	1.0
  (2, 13220)	1.12
  (3, 3061)	1.0
  (3, 13220)	1.99
  (4, 11595)	1.0
  (4, 13220)	0.45
  (5, 6114)	1.0
  (5, 13220)	12.19
  (6, 7220)	1.0
  (6, 13220)	3.39
  (7, 11511)	1.0
  (7, 13220)	6.69
  (8, 11347)	1.0
  (8, 13220)	2.34
  (9, 11391)	1.0
  (9, 13220)	5.48
  (10, 9469)	1.0
  (10, 13220)	0.9
  (11, 11267)	1.0
  (11, 13220)	2.08
  (12, 375)	1.0
  :	:
  (73895, 13220)	12.64
  (73896, 9873)	1.0
  (73896, 13220)	4.99
  (73897, 11569)	1.0
  (73897, 13220)	4.13
  (73898, 177)	1.0
  (73898, 13220)	5.05
  (73899, 10373)	1.0
  (73899, 13220)	5.33
  (73900, 3378)	1.0
  (73900, 13220)	5.18
  (73901, 1531)	1.0
  (73901, 13220)	17.13
  (73902, 5716)	1.0
  (73902, 13220)	18.18
  (73903, 12189)	1.0
  (73903, 13220)	17.63
  (73904, 8719)	1.0
  (73904, 13220)	18.36
  (73905, 11397)	1.0
  (73905, 13220)	2.5
  (73906, 3014)	1.0
  (73906, 13220)	14.48
  (73907, 600)	1.0
  (73907, 13220)	1.81


d:\Program Files\Python310\envs\mldeploy\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


d:\Program Files\Python310\envs\mldeploy\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\Program Files\Python310\envs\mldeploy\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Interacting with the model registry

In [11]:
RUN_ID = '06b0431d472841d899a8cbc5a8fc52a1'
client = MlflowClient(f"http://{TRACKING_SERVER_HOST}")

In [12]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [13]:
print(path)

C:\Users\ARUNGA~1\AppData\Local\Temp\tmpb9pfs7o5\dict_vectorizer.bin
